## AuxTel bias stability issue

Craig Lage - 25Nov22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from lsst.ip.isr import IsrTask, IsrTaskConfig
import lsst.afw.display as afwDisplay


In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/oga', collections=["LATISS/raw/all", "LATISS/calib"])


In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
# Get the timestamps of the desired image:
expId = 2022110300020
mData = butler.get('raw.metadata', detector=0, exposure=expId)
start = Time(mData['DATE-BEG'])
end = Time(mData['DATE-END'])

In [ ]:
start

In [ ]:
start = Time("2022-10-15T18:48:34")
end = Time("2022-11-03T20:48:34")

In [ ]:
test = await client.select_time_series("lsst.sal.ATCamera.focal_plane_Reb", ["*"], start, end)

In [ ]:
test.tail(2)

In [ ]:
max_slopes = []

bias = butler.get('bias', detector=0, exposure=2022110800013)
biasArr = bias.image.array
plt.title("AuxTel bias stability - Amp C10")
for expId in [2022110300020, 2022110800013, 2022110900005, 2022111000500, 2022111000600, \
             2022111000700, 2022111000804, 2022111000900, 2022111001000]:
    exp = butler.get('raw', detector=0, exposure=expId)
    biasExp = butler.get('bias', detector=0, exposure=expId)
    isrResult = isrTask.run(exp, bias=biasExp)
    arr = isrResult.exposure.image.array
    data = np.mean(arr[2100:3900, 0:400], axis=0)
    kernel_size = 10
    kernel = np.ones(kernel_size) / kernel_size
    smoothed_data = np.convolve(data, kernel, mode='same')
    deriv = np.zeros(117)
    for i in range(110, 225):
        deriv[i-110] = smoothed_data[i+1] - smoothed_data[i]
    smoothed_deriv = np.convolve(deriv, kernel, mode='same')
    peak = 110 + np.argmax(smoothed_deriv)
    plt.plot(smoothed_deriv)
    max_slopes.append([expId, peak])
    plt.plot(smoothed_data, label=f"{expId}")
    plt.ylim(-4,4)
plt.plot(np.mean(biasArr[2100:3900, 0:400], axis=0), ls='--', lw=2, color='black', label="Master bias")
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
#plt.savefig("/home/c/cslage/u/AuxTel/isr/Bias_Stability_4_Amp_C10_10Nov22.pdf")

In [ ]:
bias = butler.get('bias', detector=0, exposure=2022110800013)
biasArr = bias.image.array
plt.title("AuxTel bias stability - Amp C10")
for expId in [2022110900005, 2022110900176, 2022110800013, 2022110300020]:
    exp = butler.get('raw', detector=0, exposure=expId)
    biasExp = butler.get('bias', detector=0, exposure=expId)
    isrResult = isrTask.run(exp, bias=biasExp)
    arr = isrResult.exposure.image.array
    plt.plot(np.mean(arr[2100:3900, :], axis=0), label=f"{expId}")
    plt.ylim(-20,20)
plt.plot(np.mean(biasArr[2100:3900, :], axis=0), ls='--', lw=2, color='black', label="Master bias")
plt.legend()
plt.xlabel("X coord (pixels)")
plt.xlim(475, 509)
plt.ylabel("ADU")
#plt.savefig("/home/c/cslage/u/AuxTel/isr/Bias_Stability_2_Amp_C10_10Nov22.pdf")

In [ ]:
V26bias = butler.get('bias', detector=0, exposure=2022110800013)
V26biasArr = V26bias.image.array
V25bias = butler.get('bias', detector=0, exposure=2022091400020)
V25biasArr = V25bias.image.array

sequencer = ['V25','V26','V25','V26','V26','V26','V26']

fig = plt.figure()
ax = plt.subplot(111)
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(0.9, 0.5))
plt.title("AuxTel bias stability - Amp C10")
for i, expId in enumerate([2022091400020, 2022092700020, 2022092900020, 2022110900005, 2022110900176, \
              2022110800013, 2022110300020]):
    exp = butler.get('raw', detector=0, exposure=expId)
    biasExp = butler.get('bias', detector=0, exposure=expId)
    isrResult = isrTask.run(exp, bias=biasExp)
    arr = isrResult.exposure.image.array
    ax.plot(np.mean(arr[2100:3900, 0:400], axis=0), label=f"{expId}-{sequencer[i]}")
ax.set_ylim(-4,4)
ax.plot(np.mean(V26biasArr[2100:3900, 0:400], axis=0), ls='--', lw=3, color='black', label="V26 Master bias")
ax.plot(np.mean(V25biasArr[2100:3900, 0:400], axis=0), ls='--', lw=3, color='violet', label="V25 Master bias")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel("X coord (pixels)")
ax.set_ylabel("ADU")
fig.savefig("/home/c/cslage/u/AuxTel/isr/Bias_Stability_3_Amp_C10_10Nov22.pdf")

In [ ]:
V26bias = butler.get('bias', detector=0, exposure=2022110800013)
V26biasArr = V26bias.image.array
V25bias = butler.get('bias', detector=0, exposure=2022091400020)
V25biasArr = V25bias.image.array

sequencer = ['V25','V26','V25','V26','V26','V26','V26']

fig = plt.figure()
ax = plt.subplot(111)
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(0.9, 0.5))
plt.title("AuxTel bias stability - Amp C10")
for i, expId in enumerate([2022091400020, 2022092700020, 2022092900020, 2022110900005, 2022110900176, \
              2022110800013, 2022110300020]):
    exp = butler.get('raw', detector=0, exposure=expId)
    biasExp = butler.get('bias', detector=0, exposure=expId)
    isrResult = isrTask.run(exp, bias=biasExp)
    arr = isrResult.exposure.image.array
    ax.plot(np.mean(arr[2100:3900, :], axis=0), label=f"{expId}-{sequencer[i]}")
ax.set_ylim(-20,20)
ax.plot(np.mean(V26biasArr[2100:3900, :], axis=0), ls='--', lw=3, color='black', label="V26 Master bias")
ax.plot(np.mean(V25biasArr[2100:3900, :], axis=0), ls='--', lw=3, color='violet', label="V25 Master bias")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel("X coord (pixels)")
ax.set_ylabel("ADU")
ax.set_xlim(475,509)
fig.savefig("/home/c/cslage/u/AuxTel/isr/Bias_Stability_4_Amp_C10_10Nov22.pdf")

In [ ]:
deriv = np.zeros(117)
for i in range(110, 225):
    deriv[i-110] = smoothed_data[i+1] - smoothed_data[i]
smoothed_deriv = np.convolve(deriv, kernel, mode='same')
peak = 110 + np.argmax(smoothed_deriv)
plt.plot(smoothed_deriv)
print(peak)

In [ ]:
max_slo